# NGAME training pipeline with multi-triplet loss with clustering

In [1]:
#| default_exp 19-ngame-training-pipeline-with-multitriplet-loss-with-clustering

In [2]:
#| hide
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
#| export
import os,torch, torch.multiprocessing as mp
from xcai.basics import *
from xcai.models.MMM00X import DBT009

In [14]:
import re, json
from typing import Optional

def get_best_model(mdir:str, pat:Optional[str]=r'^checkpoint-(\d+)'):
    nm = sorted([int(re.match(pat, o).group(1)) for o in os.listdir(mdir) if re.match(pat, o)])[-1]
    fname = f'{mdir}/checkpoint-{nm}/trainer_state.json'
    with open(fname, 'r') as file: mname = json.load(file)['best_model_checkpoint']
    return f'{mdir}/checkpoint-{nm}' if mname is None else mname


In [5]:
os.environ['WANDB_MODE'] = 'disabled'

In [12]:
#| export
os.environ['CUDA_VISIBLE_DEVICES'] = '2,3'
os.environ['WANDB_PROJECT']='xc-nlg_19-ngame-training-pipeline-with-multitriplet-loss-with-clustering'

In [13]:
#| export
block = XCBlock.from_cfg('/home/aiscuser/scratch/datasets', 'data', valid_pct=0.001, tfm='xcnlg', 
                         tokenizer='distilbert-base-uncased', smp_features=[('lbl2data',1,2)])

/opt/conda/envs/ptca/lib/python3.9/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [31]:
#| export
args = XCLearningArguments(
    output_dir='/home/aiscuser/outputs/19-ngame-training-pipeline-with-multitriplet-loss-with-clustering-2-12',
    logging_first_step=True,
    per_device_train_batch_size=800,
    per_device_eval_batch_size=800,
    representation_num_beams=200,
    representation_accumulation_steps=100,
    save_strategy="steps",
    evaluation_strategy='steps',
    eval_steps=500,
    save_steps=500,
    save_total_limit=5,
    num_train_epochs=50,
    predict_with_representation=True,
    representation_search_type='BRUTEFORCE',
    adam_epsilon=1e-6,
    warmup_steps=100,
    weight_decay=0.01,
    learning_rate=2e-4,
    group_by_cluster=True,
    num_clustering_warmup_epochs=10,
    num_cluster_update_epochs=5,
    num_cluster_size_update_epochs=5,
    clustering_type='EXPO',
    minimum_cluster_size=1,
    maximum_cluster_size=300,
    target_indices_key='plbl2data_idx',
    target_pointer_key='plbl2data_data2ptr',
)

/opt/conda/envs/ptca/lib/python3.9/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [32]:
#| export
metric = PrecRecl(block.n_lbl, block.test.data_lbl_filterer, prop=block.train.dset.data.data_lbl,
                  pk=10, rk=200, rep_pk=[1, 3, 5, 10], rep_rk=[10, 100, 200])

In [33]:
#| export
bsz = max(args.per_device_train_batch_size, args.per_device_eval_batch_size)*torch.cuda.device_count()
model = DBT009.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4', bsz=bsz, tn_targ=10_000, 
                               margin=0.3, tau=0.1, n_negatives=5, apply_softmax=True)

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

Some weights of DBT009 were not initialized from the model checkpoint at sentence-transformers/msmarco-distilbert-base-v4 and are newly initialized: ['dr_layer_norm.bias', 'dr_layer_norm.weight', 'dr_projector.bias', 'dr_projector.weight', 'dr_transform.bias', 'dr_transform.weight', 'loss_fn.u', 'loss_fn.v']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
mname = '/home/aiscuser/scratch/Projects/xc_nlg/outputs/19-ngame-training-pipeline-with-multitriplet-loss-with-clustering-2-12/'

bsz = max(args.per_device_train_batch_size, args.per_device_eval_batch_size)*torch.cuda.device_count()
model = DBT009.from_pretrained(f'{mname}/{os.path.basename(get_best_model(mname))}', bsz=bsz, tn_targ=10_000, 
                               margin=0.3, tau=0.1, n_negatives=5, apply_softmax=True)

In [35]:
#| export
learn = XCLearner(
    model=model, 
    args=args,
    train_dataset=block.train.dset,
    eval_dataset=block.test.dset,
    data_collator=block.collator,
    compute_metrics=metric,
)

In [ ]:
#| export
if __name__ == '__main__':
    mp.freeze_support()
    learn.train()

In [36]:
o = learn.predict(block.test.dset)

  0%|          | 0/196 [00:00<?, ?it/s]

/opt/conda/envs/ptca/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


/opt/conda/envs/ptca/lib/python3.9/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [37]:
display_metric(o.metrics)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200,loss,runtime,samples_per_second,steps_per_second
0,30.9478,20.6773,15.6413,9.9589,30.9478,30.8732,31.9238,33.8752,25.6254,27.3457,29.3476,33.6172,25.6254,27.8314,29.4569,31.6304,38.8581,55.3431,59.9796,0.0511,125.3253,1416.433,0.886
